In [1]:
import os

In [2]:
%pwd

'/home/tousside/Documents/recrutement/cowrywise-customer-plan-abandonment/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/tousside/Documents/recrutement/cowrywise-customer-plan-abandonment'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path


In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, read_sql_file

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: str = CONFIG_FILE_PATH,
                 params_filepath: str = PARAMS_FILE_PATH,
                 schema_filepath: str = SCHEMA_FILE_PATH,
                 ):
                self.config = read_yaml(config_filepath)
                self.params = read_yaml(params_filepath)
                self.schema = read_yaml(schema_filepath)
                create_directories([self.config.artifacts_root])
                

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path)
        )
        
        return data_transformation_config

In [9]:
import os 
from mlProject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

In [17]:
class DataTransformation:
    def __init__(self, config:DataTransformationConfig):
        self.config = config
        
    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)
        
        # remove columns with missing values
        data.drop(columns=["address_city", "address_country", "last_transaction_date", "monthly_salary"],
                  inplace=True, errors='ignore')
        # replace NAN values in gender_id column by the most frequent value

        # Split the data into training and test sets. (0.75, 0.25) split.
        # create target column
        data["plan_abondonment"] = data["inactivity_days"].apply(
                lambda x: 1 if x >= 365 else 0
            )
        # drop the original inactivity_days column
        data.drop(columns=["inactivity_days"], inplace=True, errors='ignore')            
        train, test = train_test_split(data, 
                                       stratify=data["plan_abondonment"], 
                                       random_state=42)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [18]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2025-05-29 09:35:03,097: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-05-29 09:35:03,100: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-29 09:35:03,103: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-29 09:35:03,105: INFO: common: created directory at: artifacts]
[2025-05-29 09:35:03,106: INFO: common: created directory at: artifacts/data_transformation]
[2025-05-29 09:35:03,156: INFO: 4034741231: Splited data into training and test sets]
[2025-05-29 09:35:03,158: INFO: 4034741231: (2113, 9)]
[2025-05-29 09:35:03,160: INFO: 4034741231: (705, 9)]
(2113, 9)
(705, 9)
